In [1]:

from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langsmith.client import dump_model

load_dotenv()
import os

from typing import List, Optional
import os
from pathlib import Path

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

from fastapi import FastAPI, UploadFile, File
import pandas as pd
import numpy as np
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser



In [2]:
mail_raw = """
Виталий, хотел с Вами связаться по поводу гайд по п.2.6. – обсудить, т.к. не совсем понял, если честно.



Мой телефон – 8916-936-03-32



С уважением, Острик Петр



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com]
Sent: Friday, February 9, 2018 12:39 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru>
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com>; Lyovkin Boris <boris.lyovkin@bearingpoint.ru>; Katushenko Kate <kate.katushenko@bearingpoint.ru>
Subject: RE: ПУТ (ТМ014)



Добрый день, Петр!



Спасибо за примеры, проверил на них удаление лота. Теперь должно быть правильно.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru]
Sent: Friday, February 09, 2018 1:47 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



Добрый день!

Примеры

41…703

41…704

41…705



Сейчас по 2.6 отпишусь







From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Thursday, February 8, 2018 8:03 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Добрый день, Петр!



У нас было три пункта для исправлений.

По релевантным перевозчикам изменения в логику внес, этот пункт можно протестировать.

По ошибке с удалением лота, с виду должно быть в порядке, но чтобы удостовериться, нужны примеры. Сейчас в ПУТе остался всего один ФЗ. На одном ФЗ этот пункт нормально протестировать не получится. Можете сделать побольше тестовых данных?

По поводу п.2.6., вы предлагали использовать один из двух BADI (ранее в переписке), хорошо, но я плохо представляю как эта часть работает в режиме пользователя. Вы можете сделать мини инструкцию по этой части с примером? Так я смогу проверить логику перед передачей в тест.



Итого нужны новые тестовые данные в ПУТ и гайд по п.2.6.

А тем временем можете проверить проверку на перевозчиков.



Спасибо.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru]
Sent: Thursday, February 08, 2018 5:46 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



Добрый день!

Хотел у вас уточнить статус по тм014, когда можно продолжать тестировать с исправлениями?



С уважением, Острик Петр







From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Monday, January 29, 2018 3:05 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Добрый день!



Хорошо, буду это учитывать.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru]
Sent: Monday, January 29, 2018 5:01 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



Виталий,

Добрый День!



По поводу пункта :

По итогам тендера не выполняется проверка п.2.6 -

Лот может быть выигран только целиком, иначе он расформировывается.

В нашем примере 1 фз выиграл один перевозчик, два другой, соответственно, они и присвоились (ошибка).



Здесь достаточно выводить предупреждение, что тендер Лотом не состоялся, Логист должен сам вручную расформировать лот.



С уважением, Острик Петр





From: Ostrik Petr
Sent: Wednesday, January 24, 2018 8:10 PM
To: 'Pristupov Vitaliy' <Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Добрый день, Виталий!



Вы правы, если находясь в пульте вызвать список экспедиторов и при этом не сохраниться, то список не попадет в талблицы и соответственно проверка не будет успешной. Для устранения, соответственно, необходимо брать данные из ПУТа до сохранения.



Если выделить ФЗ лота и нажать – удалить Лот, то он пропадет только из самого верхнего ФЗ.



По итогам тендера не выполняется проверка п.2.6 -

Лот может быть выигран только целиком, иначе он расформировывается.

В нашем примере 1 фз выиграл один перевозчик, два другой, соответственно, они и присвоились (ошибка).



С уважением, Острик Петр



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com]
Sent: Wednesday, January 24, 2018 7:21 AM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Добрый день, Петр!



Посмотрел ваш пример.

Он вроде бы и корректный, но тут есть кое что странное.

Вы прислали скрин, на котором экспедиторы существуют, однако, например, для ФЗ 4100000662 (ключ объекта 005056B45C901ED7BFEEE69DDC3160D0) в таблице /SCMTMS/D_TORRL (что соответствует узлу TOR.RANKINGLIST) нет к ней записей.

Если попробовать посмотреть этот же ФЗ через транзакцию BOBT, то связка с экспедиторами также отсутствует.

Опираясь на все это, логика программы считает, что экспедиторов нет.

Можете проверить, действительно ли экспедиторы привязаны к ФЗ?



Спасибо.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru]
Sent: Tuesday, January 23, 2018 5:16 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



Добрый день!



Примеры составлены корректно?



С уважением, Острик Петр



From: Ostrik Petr
Sent: Monday, January 22, 2018 4:57 PM
To: 'Pristupov Vitaliy' <Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Добрый день,



Не работает создание лота – проблема с проверкой перевозчиков.

Во всех трех заказах из примера – одинаковый набор перевозчиков.



Однако лот не создается. Пишет







Пример в TPR610





С уважением, Острик Петр



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com]
Sent: Monday, January 15, 2018 5:57 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Проверки в ПУТ доработал, можно попробовать еще раз.

По поводу отсутствия номера лота на портале, средствами ABAP можно добавить поле только в конец таблицы, оно сейчас там есть (закладка Запросы фрахта, скрин ниже).

Дамп на портале исправил.







С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru]
Sent: Monday, January 15, 2018 7:23 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: FW: ПУТ (ТМ014)



UPD. А еще в дамп упало













From: Ostrik Petr
Sent: Monday, January 15, 2018 5:11 PM
To: 'Pristupov Vitaliy' <Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Добрый день, по поводу проверки – подкорректировал немного, желтым выделил.



По – поводу теста – не смог сформировать *632, *633 выдает



Не могу найти поле с лотом на портале



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com]
Sent: Monday, January 15, 2018 3:52 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Петр, в общем, можете начинать тестировать.

Отклонения возможны, поэтому если что то пойдет не так, то присылайте конкретный пример с указанием набора ФЗ и конкретной операции, которая дала сбой. Это ускорит мне работу.

Либо если нужны корректировки логики, то тоже указывайте. Спорных моментов можно придумать много.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Monday, January 15, 2018 12:41 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



Добрый день!

У меня в ПУТ пропали этапы ЕФ (там горизонт планирования 5 дней был), а ЗФ на месте.

У вас что конкретно пропало?





С уважением, Острик Петр





From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Saturday, January 13, 2018 12:54 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Петр, добрый день!



Хотел сегодня поотлаживать функции, но из ПУТа кажется исчезли все данные. Это так и должно быть?



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Friday, January 12, 2018 5:35 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



Добрый день, попытайтесь сейчас, а вообще я нашел неточность в своей спеке, сейчас подправлю



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Friday, January 12, 2018 1:58 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Петр, сейчас при попытке отправить ФЗ 4100000621 на конкурс вижу ошибку, что он не в статусе 03.

У нее есть лот, поэтому я хотел попробовать отправить этот ФЗ на портал.

Можно как то поправить ошибку со статусом?



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Friday, January 12, 2018 3:08 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)





Если не появляется убедитесь что запрос отправлен







From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Friday, January 12, 2018 12:47 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Хорошо, получилось отправить на конкурс один ФЗ, где теперь его можно увидеть на портале?



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Friday, January 12, 2018 2:42 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



Попробуйте пожалуйста те же, я разблокировал ДП zcarrier01, если вы об этом.

С уважением, Острик Петр



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Friday, January 12, 2018 12:41 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Петр, можете предоставить еще пример ФЗ, который можно отправить на конкурс без ошибок (где не блокирован ДП)?



Спасибо.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Friday, January 12, 2018 12:18 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



По второму пункту наврал! Можем отправить и лоты и единичные ФЗ, просто при этом для лота должна осуществляться проверка полноты выделения.



From: Ostrik Petr
Sent: Friday, January 12, 2018 10:15 AM
To: 'Pristupov Vitaliy' <Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Добрый день!



1.       Да, так можно

2.       Да, вы правы, получается оправляем либо только лоты отдельно, либо только единичные ФЗ



С уважением, Острик Петр



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Friday, January 12, 2018 8:34 AM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Добрый день, Петр!



Пара уточняющих вопросов по отправке на конкурс:

1.       Если выделены ФЗ, присвоенные трем разным лотам, но по каждому лоту при этом выделены все ФЗ, то так можно отправить на конкурс?

2.       Если пытаемся отправить на конкурс ФЗ без присвоенного лота, это корректно? Если да, то при этом выходит должны быть выделены только ФЗ без лота.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Thursday, January 11, 2018 7:23 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



Если есть возможность встройки проверки для стандартной кнопки, то отдельная кнопка не нужна. Но при выделении одного ФЗ из списка лота при отправке выдавать сообщение “Для Лота N XXX не выбраны все ФЗ”.

Причем если выбраны два ФЗ из двух разных лотов, то дважды вывести данное сообщение (для каждого лота соотвественно).

С уважением, Острик Петр



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Thursday, January 11, 2018 5:05 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



А раз это она и есть, то тогда я правильно понимаю, что мы создаем новую кнопку только ради одной проверки?



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Thursday, January 11, 2018 6:51 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris; Katushenko Kate
Subject: RE: ПУТ (ТМ014)



Да, это она и имеется ввиду.



С уваженем, Острик Петр



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Thursday, January 11, 2018 2:06 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >; Katushenko Kate <kate.katushenko@bearingpoint.ru <mailto:kate.katushenko@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



Петр, еще вопрос для понимания – у нас планируется кнопка «Отправить лот на конкурс».

Эта кнопка как то связана со стандартной кнопкой «Конкурс->Запуск»?

Просто в спеке в п.2.5. коротко указано «Запустить конкурс и создать запросы фрахта…».

Если кнопки не связаны, то тогда раскройте немного что должно происходить по этой кнопке помимо проверок запуска конкурса. В чем заключается запуск конкурса.







С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Thursday, January 11, 2018 1:35 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris
Subject: RE: ПУТ (ТМ014)



Да, именного так - каждый с каждым из выбранного набора ФЗ.



С уважением, Острик Петр



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Thursday, January 11, 2018 11:34 AM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >; Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >
Subject: RE: ПУТ (ТМ014)



А, понятно, то есть проверяем не любой, а каждый с каждым до тех пор, пока не останутся только общие для всех перевозчики.

И уже по результату делать вывод создавать лот или нет.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Thursday, January 11, 2018 12:27 PM
To: Pristupov Vitaliy
Cc: Noskova Kseniya; Lyovkin Boris
Subject: RE: ПУТ (ТМ014)



Добрый день, Виталий!



В том и был замысел. Берется набор ФЗ, и если в каких-то ФЗ в ранговых списках нет общего перевозчика, то он выкидывается и у остальных тоже. Остаются только те перевозчики, которые являются общими для данной выборки.



С уважением, Острик Петр



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Thursday, January 11, 2018 9:22 AM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Noskova Kseniya <Kseniya.Noskova@bcsoft.com <mailto:Kseniya.Noskova@bcsoft.com> >
Subject: RE: ПУТ (ТМ014)



Добрый день, Петр!



Возник вопрос по п.2.3.2. спеки.

В нем сказано, что для проверки подойдет любой ФЗ и на его основе проводится проверка наличия перевозчиков.

Но тогда если я, например, захочу добавить в один лот сразу 10 ФЗ (по выделению), то по каким то из них общий перевозчик будет существовать, а по каким то нет.

Соответственно и проверка может отработать не корректно.

Другими словами, брать случайный ФЗ для проверки несколько странно.

Или все таки все окей?



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Wednesday, January 10, 2018 8:21 PM
To: Pristupov Vitaliy
Cc: Lyovkin Boris
Subject: RE: ПУТ



Теперь работает.



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Wednesday, January 10, 2018 6:19 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >
Subject: RE: ПУТ



Хм, интересно, а попробуйте теперь.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Стахановская, 54Б, Пермь, Россия, 614066



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Wednesday, January 10, 2018 8:10 PM
To: Pristupov Vitaliy
Cc: Lyovkin Boris
Subject: RE: ПУТ



Виталий, у нас опять дампит ПУТ. Вы работаете?

С уважением, Острик Петр





From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Wednesday, January 10, 2018 12:25 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Cc: Lyovkin Boris <boris.lyovkin@bearingpoint.ru <mailto:boris.lyovkin@bearingpoint.ru> >
Subject: RE: ПУТ



Кажется починилось, по крайней мере у меня по данному сценарию что то создалось (надеюсь ничего не запорол при этом).



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Клары Цеткин, 10А, Пермь, Россия, 614010



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Wednesday, January 10, 2018 2:14 PM
To: Pristupov Vitaliy
Cc: Lyovkin Boris
Subject: RE: ПУТ



Падает,

ПУТ-> выделить ЕФ-> создать документ мощности -> дамп



From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Wednesday, January 10, 2018 12:07 PM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Subject: RE: ПУТ



Ну даже не знаю, сейчас я выключил все свои наработки для чистоты эксперимента.

Попробуйте еще раз.

Если все равно будет падать, то вышлите мне последовательность шагов для повторения ошибки.

С ПУТ больше никто не работает случаем?



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Клары Цеткин, 10А, Пермь, Россия, 614010



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Wednesday, January 10, 2018 1:58 PM
To: Pristupov Vitaliy
Subject: RE: ПУТ



Падает





From: Pristupov Vitaliy [mailto:Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com> ]
Sent: Wednesday, January 10, 2018 11:57 AM
To: Ostrik Petr <petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> >
Subject: RE: ПУТ



Петр, сейчас дампов нет?

Если все хорошо, то прошу написать, когда я могу продолжить работу.

Спасибо.



С уважением,

Виталий Приступов

Разработчик

Т + 7 342 258 00 62 доб. 473 | Vitaliy.Pristupov@bcsoft.com <mailto:Vitaliy.Pristupov@bcsoft.com>

АО БК Софт | ул. Клары Цеткин, 10А, Пермь, Россия, 614010



From: Ostrik Petr [mailto:petr.ostrik@bearingpoint.ru <mailto:petr.ostrik@bearingpoint.ru> ]
Sent: Wednesday, January 10, 2018 12:55 PM
To: Pristupov Vitaliy
Cc: Lyovkin Boris
Subject: ПУТ



Добрый день!

Мне очень нужно сделать пример для программиста, но падает в дамп при планировании в ПУТ. Полагаю, это из-за того, что Вы с ним работаете сейчас?

Могли бы вы отпустить ПУТ ненадолго?



С уважением, Острик Петр


"""

In [3]:
llm = init_chat_model("gpt-4.1-mini", model_provider="openai")

In [12]:
class MailTopic(BaseModel):
    topic_name: str = Field(..., description="тема содержания mail")
    topic_description: str = Field(..., description=" краткое описание темы mail")
    topic_value: int = Field(..., description="важность письмо по шкале от 0 до 10")


class MailSearch(BaseModel):
   cc_persons: List[str] = Field(None, description="список всех mail адресов в копии письма")
   companies: List[str] = Field(None, description="корпоративные префиксы mail адресов участников в копии письма")
   mail_topics: List[MailTopic] = Field(..., description="различные темы содержания email")

parser = PydanticOutputParser(pydantic_object=MailSearch)

chain = llm | parser



In [17]:
output_search_answer = chain.invoke(mail_raw + str(parser.get_format_instructions()))

In [19]:
output_search_answer.model_dump_json()

'{"cc_persons":["Kseniya.Noskova@bcsoft.com","boris.lyovkin@bearingpoint.ru","kate.katushenko@bearingpoint.ru"],"companies":["bcsoft.com","bearingpoint.ru"],"mail_topics":[{"topic_name":"Уточнение по гайд по п.2.6.","topic_description":"Виталий запрашивает гайд и разъяснения по пункту 2.6. из спецификации, так как не совсем понял логику и просит обсудить детали.","topic_value":8},{"topic_name":"Проверка и исправление логики удаления лота","topic_description":"Обсуждение исправлений в логике удаления лота и проверка на примерах от Петра, чтобы убедиться в корректности реализованных изменений.","topic_value":7},{"topic_name":"Проверка на общих перевозчиков при создании лота","topic_description":"Обсуждение алгоритма проверки наличия общих перевозчиков для группы ФЗ, чтобы корректно создавать лоты с учетом общих экспедиторов для всего набора заказов.","topic_value":9},{"topic_name":"Проблемы с тестовыми данными для проверки ФЗ и лотов","topic_description":"Запрос от Виталия дополнительных

In [ ]:
видео по декораторам, по ООП

In [7]:
llm.invoke(mail_raw)

AIMessage(content='Добрый день, Петр!\n\nПо п.2.6.:\n\nСуть пункта в том, что лот должен выигрывать целиком, не допускается ситуация, когда один перевозчик выигрывает часть заявок (ФЗ) в лоте, а другой — другую часть. Если это происходит, лот считается несостоявшимся и должен быть расформирован.\n\nЧто предлагается реализовать:\n\n- При подведении итогов тендера (конкурса) проверять для каждого лота, что все ФЗ лота выиграны одним и тем же перевозчиком.\n- Если обнаруживается, что один лот «разбит» между несколькими перевозчиками, выводить предупреждение («Лот N не выиграл целиком, пожалуйста, расформируйте лот вручную»).\n- Логика автоматического расформирования лота не реализуется, т.к. предпочтительно, чтобы логист вручную принял решение.\n- В документации или инструкции предлагается описать, что по п.2.6. функционал проверки должен выбрасывать предупреждение, а дальнейшие действия — за пользователем.\n\n---\n\n**Как это работает в режиме пользователя**\n\nПользователь (логист) запу